In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
import calendar
import pandas  as pd 
from tqdm import tqdm

1. extract restaurant url from each region in jakarta.
2. save as txt 

In [2]:
def extract_main_page(url):
#     url = 'https://www.zomato.com/jakarta/kelapa-gading-restaurants'
    driver = webdriver.Chrome(executable_path='chromedriver.exe')

    chrome_options = Options()
    chrome_options.add_argument("--disable-popup-blocking")
    driver.get(url)
    
    time.sleep(3)
    # scroll until the end of the page
    
    # method 1 
#     page = driver.find_element_by_tag_name("html")
#     page.send_keys(Keys.END)

#     time.sleep(120)
    
    # method 2 
    
    last_height = 0
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollBy({}, {});".format(last_height,last_height+100))
        # Wait to load page
        time.sleep(5)
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
#         print(last_height, new_height,'ok')
        if new_height == last_height:
            break
        last_height = new_height
    
    # get all restaurants url 
    lnks=driver.find_elements_by_xpath("//a[@href]")
    restaurant_link = [] 

    for lnk in lnks:
        if 'info' in lnk.get_attribute("href"): 
            restaurant_link.append(lnk.get_attribute("href"))
        else : 
            pass 
    restaurant_link = list(set(restaurant_link))
    
    driver.close()
    return restaurant_link

In [13]:
df = pd.read_csv('data/0_jak_area.txt')
df = df.tail(6)

In [14]:
df.tail(6)

,url
19,https://www.zomato.com/jakarta/beji-restaurants
20,https://www.zomato.com/jakarta/bogor-timur-res...
21,https://www.zomato.com/jakarta/bogor-tengah-re...
22,https://www.zomato.com/jakarta/bekasi-utara-re...
23,https://www.zomato.com/jakarta/pondok-aren-res...
24,https://www.zomato.com/jakarta/bekasi-selatan-...


In [15]:
df['url'].loc[0]

KeyError: 0

In [16]:
restaurant_list = []
for item in tqdm(df.itertuples()):
    tmp = {} 
    tmp['url'] = item.url
    tmp['link'] = extract_main_page(item.url)
    restaurant_list.append(tmp)
    time.sleep(3)

6it [10:29, 104.89s/it]


In [5]:
# restaurant_list[0]

In [17]:
for item in restaurant_list:
    print(item.keys(), len(list(item.values())[0]))

dict_keys(['url', 'link']) 47
dict_keys(['url', 'link']) 54
dict_keys(['url', 'link']) 55
dict_keys(['url', 'link']) 55
dict_keys(['url', 'link']) 54
dict_keys(['url', 'link']) 57


In [9]:
# for key, value in restaurant_list.items():
#     print (key, len(value))

In [18]:
tes = pd.DataFrame(restaurant_list)
tes['total'] = tes['link'].apply(len)

In [19]:
tes['total'].sum()

2502

In [20]:
tes

,url,link,total
0,https://www.zomato.com/jakarta/beji-restaurants,[https://www.zomato.com/jakarta/kopium-artisan...,501
1,https://www.zomato.com/jakarta/bogor-timur-res...,[https://www.zomato.com/jakarta/sop-buntut-ibu...,297
2,https://www.zomato.com/jakarta/bogor-tengah-re...,[https://www.zomato.com/jakarta/mie-ayam-bakso...,369
3,https://www.zomato.com/jakarta/bekasi-utara-re...,[https://www.zomato.com/jakarta/mie-ayam-hotpl...,501
4,https://www.zomato.com/jakarta/pondok-aren-res...,[https://www.zomato.com/jakarta/pasta-pasti-po...,501
5,https://www.zomato.com/jakarta/bekasi-selatan-...,[https://www.zomato.com/jakarta/beefbox-bekasi...,333


In [21]:
tes.iloc[2]['url']

'https://www.zomato.com/jakarta/bogor-tengah-restaurants'

In [22]:
tes.to_pickle('data/restaurant_raw2_extra.pickle')

In [ ]:
tes.to_csv('data/1_bodebek.csv')